In [174]:
import copy

import pandas as pd
import numpy as np
import more_itertools
import csv
import random
import itertools
pd.options.mode.chained_assignment = None  # default = 'warn'

In [74]:
product_rules = pd.read_csv('Data/master_rules.csv')
# product_rules.drop(columns=['Unnamed: 0'], inplace=True)
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,Instant food products,soda,0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,Instant food products,whole milk,0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,UHT-milk,whole milk,0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,abrasive cleaner,rolls/buns,0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,artif. sweetener,other vegetables,0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [160]:
with open('Data/names.csv') as f:
    read = csv.reader(f)
    names = list(read)

In [161]:
print(names)

[['Instant food products', 'UHT-milk', 'abrasive cleaner', 'artif. sweetener', 'baby cosmetics', 'bags', 'baking powder', 'bathroom cleaner', 'beef', 'berries', 'beverages', 'bottled beer', 'bottled water', 'brandy', 'brown bread', 'butter', 'butter milk', 'cake bar', 'candles', 'candy', 'canned beer', 'canned fish', 'canned fruit', 'canned vegetables', 'cat food', 'cereals', 'chewing gum', 'chicken', 'chocolate', 'chocolate marshmallow', 'citrus fruit', 'cleaner', 'cling film/bags', 'cocoa drinks', 'coffee', 'condensed milk', 'cooking chocolate', 'cookware', 'cream', 'cream cheese ', 'curd', 'curd cheese', 'decalcifier', 'dental care', 'dessert', 'detergent', 'dish cleaner', 'dishes', 'dog food', 'domestic eggs', 'female sanitary products', 'finished products', 'fish', 'flour', 'flower (seeds)', 'flower soil/fertilizer', 'frankfurter', 'frozen chicken', 'frozen dessert', 'frozen fish', 'frozen fruits', 'frozen meals', 'frozen potato products', 'frozen vegetables', 'fruit/vegetable jui

In [163]:
print(len(names[0]))

167


In [75]:
product_rules.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object

In [88]:
test = product_rules['antecedents'].iloc[0]
print(test)
test1 = product_rules['antecedents'].iloc[167]
print(test1)

Instant food products
UHT-milk, curd


In [89]:
def change_format(x):
    delim = ','
    return frozenset(x.split(delim))

In [90]:
test = change_format(test)
print(test)
print(type(test))
test1 = change_format(test1)
print(test1)
print(type(test1))

frozenset({'Instant food products'})
<class 'frozenset'>
frozenset({' curd', 'UHT-milk'})
<class 'frozenset'>


In [91]:
product_rules['antecedents'] = product_rules['antecedents'].apply(change_format)
product_rules['consequents'] = product_rules['consequents'].apply(change_format)
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [92]:
product_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
...,...,...,...,...,...,...,...,...,...
24689,"( root vegetables, soda, other vegetables, s...","( rolls/buns, whole milk)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069
24690,"( root vegetables, other vegetables, sausage,...","(soda, rolls/buns)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124
24691,"(rolls/buns, root vegetables, soda, sausage...","( other vegetables, whole milk)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189
24692,"(rolls/buns, root vegetables, sausage, whol...","(soda, other vegetables)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947


In [93]:
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121


In [122]:
product_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
...,...,...,...,...,...,...,...,...,...
24689,"( root vegetables, soda, other vegetables, s...","( rolls/buns, whole milk)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069
24690,"( root vegetables, other vegetables, sausage,...","(soda, rolls/buns)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124
24691,"(rolls/buns, root vegetables, soda, sausage...","( other vegetables, whole milk)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189
24692,"(rolls/buns, root vegetables, sausage, whol...","(soda, other vegetables)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947


In [157]:
product_rules.tail()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
24689,"( root vegetables, soda, other vegetables, s...","( rolls/buns, whole milk)",0.004876,0.178599,0.002566,0.526316,2.946915,0.001695,1.734069
24690,"( root vegetables, other vegetables, sausage,...","(soda, rolls/buns)",0.004876,0.119836,0.002566,0.526316,4.391976,0.001982,1.858124
24691,"(rolls/buns, root vegetables, soda, sausage...","( other vegetables, whole milk)",0.004106,0.191429,0.002566,0.625000,3.264913,0.001780,2.156189
24692,"(rolls/buns, root vegetables, sausage, whol...","(soda, other vegetables)",0.004362,0.124198,0.002566,0.588235,4.736266,0.002024,2.126947
24693,"( root vegetables, bottled water, soda, saus...","( rolls/buns, other vegetables)",0.005132,0.146780,0.002566,0.500000,3.406469,0.001813,1.706441


In [95]:
product_rules.dtypes

antecedents            object
consequents            object
antecedent support    float64
consequent support    float64
support               float64
confidence            float64
lift                  float64
leverage              float64
conviction            float64
dtype: object

In [127]:
query = product_rules[(product_rules['antecedents'].astype(str).str.contains('UHT-milk')) & (product_rules['antecedents'].astype(str).str.contains('beverages'))]
query

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
141,"(beverages, UHT-milk)",(other vegetables),0.006159,0.376700,0.003336,0.541667,1.437926,0.001016,1.359927
142,"(beverages, UHT-milk)",(whole milk),0.006159,0.458045,0.004106,0.666667,1.455462,0.001285,1.625866


In [128]:
query1 = product_rules[product_rules['antecedents'].astype(str).str.contains('UHT-milk' and 'beverages')]
query1

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
141,"(beverages, UHT-milk)",(other vegetables),0.006159,0.376700,0.003336,0.541667,1.437926,0.001016,1.359927
142,"(beverages, UHT-milk)",(whole milk),0.006159,0.458045,0.004106,0.666667,1.455462,0.001285,1.625866
310,"( beef, beverages)",(other vegetables),0.008981,0.376700,0.005645,0.628571,1.668626,0.002262,1.678115
311,"( beef, beverages)",(whole milk),0.008981,0.458045,0.005389,0.600000,1.309916,0.001275,1.354888
360,"(beef, misc. beverages)",(whole milk),0.008211,0.458045,0.004106,0.500000,1.091597,0.000345,1.083911
...,...,...,...,...,...,...,...,...,...
22597,"( whole milk, tropical fruit, rolls/buns, mi...",(other vegetables),0.003849,0.376700,0.002566,0.666667,1.769755,0.001116,1.869900
22598,"(yogurt, other vegetables, rolls/buns, misc...",(whole milk),0.004619,0.458045,0.003079,0.666667,1.455462,0.000964,1.625866
22599,"( rolls/buns, other vegetables, whole milk, ...",(yogurt),0.005902,0.282782,0.003079,0.521739,1.845025,0.001410,1.499638
22600,"( misc. beverages, other vegetables, whole mi...",(rolls/buns),0.005645,0.349756,0.003079,0.545455,1.559528,0.001105,1.430536


In [129]:
query2 = product_rules[product_rules['antecedents'].astype(str).str.contains('UHT-milk' and 'berries')]
query2

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
139,"( UHT-milk, berries)",(other vegetables),0.007442,0.376700,0.004106,0.551724,1.464625,0.001302,1.390438
140,"( UHT-milk, berries)",(whole milk),0.007442,0.458045,0.003849,0.517241,1.129238,0.000441,1.122622
309,"( beef, berries)",(whole milk),0.011291,0.458045,0.006159,0.545455,1.190833,0.000987,1.192302
412,"( beverages, berries)",(other vegetables),0.006159,0.376700,0.003593,0.583333,1.548535,0.001273,1.495920
413,"( beverages, berries)",(whole milk),0.006159,0.458045,0.003336,0.541667,1.182563,0.000515,1.182448
...,...,...,...,...,...,...,...,...,...
17077,"( whole milk, soda, yogurt, berries)",(shopping bags),0.006415,0.168335,0.003336,0.520000,3.089085,0.002256,1.732636
17078,"( whole milk, shopping bags, yogurt, berries)",(soda),0.004619,0.313318,0.003336,0.722222,2.305078,0.001889,2.472055
17079,"( shopping bags, yogurt, berries)","(soda, whole milk)",0.005645,0.150885,0.003336,0.590909,3.916280,0.002484,2.075614
17080,"( soda, tropical fruit, yogurt, berries)",(whole milk),0.004106,0.458045,0.002823,0.687500,1.500945,0.000942,1.734257


In [132]:
query3 = product_rules[(product_rules['antecedents'].astype(str).str.contains('UHT-milk')) & (product_rules['antecedents'].astype(str).str.contains('berries'))]
query3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
139,"( UHT-milk, berries)",(other vegetables),0.007442,0.376700,0.004106,0.551724,1.464625,0.001302,1.390438
140,"( UHT-milk, berries)",(whole milk),0.007442,0.458045,0.003849,0.517241,1.129238,0.000441,1.122622


In [153]:
query3.sort_values(by=['support','confidence','lift'],inplace=True)
query3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
140,"( UHT-milk, berries)",(whole milk),0.007442,0.458045,0.003849,0.517241,1.129238,0.000441,1.122622
139,"( UHT-milk, berries)",(other vegetables),0.007442,0.376700,0.004106,0.551724,1.464625,0.001302,1.390438


In [154]:
query3['consequents'] = query3['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
query3

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
140,"( UHT-milk, berries)",whole milk,0.007442,0.458045,0.003849,0.517241,1.129238,0.000441,1.122622
139,"( UHT-milk, berries)",other vegetables,0.007442,0.376700,0.004106,0.551724,1.464625,0.001302,1.390438


In [155]:
recommend = list(query3['consequents'])
print(recommend)

['whole milk', 'other vegetables']


In [156]:
product_rules.head()
product_rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,24694.000000,24694.000000,24694.000000,24694.000000,24694.000000,24694.000000,2.469400e+04
mean,0.007474,0.383971,0.004419,0.614172,1.667469,0.001476,inf
std,0.008207,0.075710,0.004397,0.095497,0.457445,0.000872,NaN
min,0.002566,0.066975,0.002566,0.500000,1.091597,0.000215,1.083911e+00
25%,0.004619,0.349756,0.002823,0.538462,1.397175,0.001015,1.365512e+00
50%,0.005645,0.376700,0.003336,0.590909,1.561548,0.001310,1.536940e+00
75%,0.007698,0.458045,0.004619,0.666667,1.804921,0.001721,1.842648e+00
max,0.376700,0.458045,0.191429,1.000000,8.128911,0.020845,inf


In [158]:
def recommend(z):
    global product_rules
    if type(z) == list:
        if len(z) == 1:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))]
        elif len(z) == 2:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))]
        elif len(z) == 3:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[2])))]
        elif len(z) == 4:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[2])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[3])))]
        elif len(z) == 5:
            master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z[0])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[1])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[2])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[3])))
                                         & (product_rules['antecedents'].astype(str).str.contains(str(z[4])))]
        else:
            return "Something Happened That Should Never Have Happened!!!"
    else:
        master_query = product_rules[(product_rules['antecedents'].astype(str).str.contains(str(z)))]
    master_query.sort_values(by=['support','confidence','lift'],inplace=True,ascending=(False,False,False), ignore_index=True)
    # maybe update possible graph here ???
    master_query['consequents'] = master_query['consequents'].apply(lambda x: ', '.join(list(x))).astype("unicode")
    result = list(master_query['conseqents'])
    return result

In [183]:
def start_here(x):
    # hold = more_itertools.powerset(x)
    # for result in more_itertools.powerset(x):
    #     print(result)
    # new_list = [combo for r in range(len(x) + 1) for combo in itertools.combinations(x, r)]
    # print(new_list)
    s = len(x)
    masks = [1 << i for i in range(s)]
    for i in range(1 << s):
        yield [ss for mask, ss in zip(masks, x) if i & mask]

In [184]:
product_rules.head(10)

In [182]:
random_index = random.randrange(len(names[0]))
list_to_check = copy.copy(names[0][random_index])
list_to_check_1 = [x[random_index] for x in names[0]] * 5
print(list_to_check_1)
print(type(list_to_check_1))
# print(list(start_here(list_to_check)))

In [150]:
print(recommend)
print(type(recommend))
print(type(recommend[0]))

[frozenset({'whole milk'})]
<class 'list'>
<class 'frozenset'>


In [133]:
product_rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Instant food products),(soda),0.015396,0.313318,0.007955,0.516667,1.649017,0.003131,1.420722
1,(Instant food products),(whole milk),0.015396,0.458045,0.007955,0.516667,1.127983,0.000903,1.121287
2,(UHT-milk),(whole milk),0.078522,0.458045,0.040544,0.516340,1.127270,0.004577,1.120529
3,(abrasive cleaner),(rolls/buns),0.005645,0.349756,0.002823,0.500000,1.429567,0.000848,1.300488
4,(artif. sweetener),(other vegetables),0.007442,0.376700,0.003849,0.517241,1.373086,0.001046,1.291121
